
# Classifying Fashion-MNIST
Now it's your turn to build and train a neural network. You'll be using the Fashion-MNIST dataset, a drop-in replacement for the MNIST dataset. MNIST is actually quite trivial with neural networks where you can easily achieve better than 97% accuracy. Fashion-MNIST is a set of 28x28 greyscale images of clothes. It's more complex than MNIST, so it's a better representation of the actual performance of your network, and a better representation of datasets you'll use in the real world.



In this notebook, you'll build your own neural network. For the most part, you could just copy and paste the code from Part 3, but you wouldn't be learning. It's important for you to write the code yourself and get it to work. Feel free to consult the previous notebooks though as you work through this.

First off, let's load the dataset through torchvision.


In [25]:
import pandas as pd
import numpy as np
import torch
from torchvision import datasets, transforms
#import helper

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
#trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
df_train = pd.read_csv('C:/Sally felsokning/T4D/train_Limbo.csv')
df_test = pd.read_csv('C:/Sally felsokning/T4D/test_Limbo.csv')

#trainloader = torch.utils.data.DataLoader(trainset, batch_size=12, shuffle=True)

# Download and load the test data
#testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)

#testloader = torch.utils.data.DataLoader(testset, batch_size=12, shuffle=True)
df_train.head(3)

,Unit,Date,Station,OA_Pillar,OSD_Pillar,Rx3Alignment,OA_EDMRange,OSD_EDMRange,SignalDependency,RatioPosPreAd,...,Range+1,Range+3,Range+5,Range+7,Range+10,Range+12,Range+14,Range+16,Range+19,Range+20
0,58002032 00305,2021-01-29 09:38:37.310,Sally_OptSigDepTest1,12,23,3128856,3128858,3129490,3129488,9.930999,...,564.618595,582.124822,588.100654,595.737933,611.887556,600.448831,613.276866,581.538323,532.311776,479.975336
1,58002032 00320,2021-01-14 10:19:12.110,Sally_OptSigDepTest1,11,23,3108472,3108474,3108785,3108783,11.257180,...,794.694505,792.812436,807.565658,819.846036,826.716099,817.006249,802.547055,748.319275,673.421546,601.009384
2,58002032 00350,2021-03-24 13:54:06.217,Sally_OptSigDepTest1,15,21,3182843,3182845,3183425,3183423,10.053430,...,763.482528,765.409295,773.055427,802.455055,785.113827,747.809394,693.010823,641.284946,559.562817,480.433346


In [32]:
#min ruta
data = df_train.copy()
test_data = df_test.copy()
names = data.columns
data = data.drop(['Unit','Date','Station', 'Rx3Alignment','OA_EDMRange','OSD_EDMRange','SignalDependency'], axis=1)
test_data = test_data.drop(['Unit','Date','Station'], axis=1)
data.OA_Pillar = data.OA_Pillar*0.01
data.OSD_Pillar = data.OSD_Pillar*0.01
test_data.OA_Pillar = test_data.OA_Pillar*0.01
#print(test_data.head(3))
test_data.OSD_Pillar = test_data.OSD_Pillar*0.01
data.AdjIt = np.where(data.AdjIt == 0,0,1) #0==Pass, 1==FAIL
test_data.AdjIt = np.where(test_data.AdjIt == 0,0,1)
quant_names = data.columns.drop(['OA_Pillar','OSD_Pillar'])
#print(names)
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_names:
    mean, std = data[each].mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean)/std
# Separate the data into features and targets
target_fields = ['AdjIt']
features, targets = data.drop(target_fields, axis=1), data[target_fields]
test_features, test_targets = test_data.drop(target_fields, axis=1), test_data[target_fields]
features.head()

,OA_Pillar,OSD_Pillar,RatioPosPreAd,RatioNegPreAd,Rx3-19,Rx3-16,Rx3-14,Rx3-7,Rx3-1,Rx3+1,...,Range+1,Range+3,Range+5,Range+7,Range+10,Range+12,Range+14,Range+16,Range+19,Range+20
0,0.12,0.23,1.343772,0.525621,-0.823409,-0.967135,-0.818717,-0.665128,-0.752173,-0.767979,...,-0.376137,-0.185460,-0.142130,-0.110505,0.022342,-0.133555,0.209911,0.228968,0.485069,0.834712
1,0.11,0.23,2.024908,0.508271,1.655322,0.884358,0.475810,0.344873,0.513822,0.537323,...,2.450749,2.419620,2.602829,2.737443,2.846409,2.870437,2.989079,2.678010,2.381329,2.334422
2,0.15,0.21,1.406653,-1.219028,0.614782,0.208199,0.198666,0.704215,1.031662,1.011215,...,2.067255,2.080789,2.171192,2.516440,2.299519,1.910568,1.380693,1.106298,0.851273,0.840387
3,0.11,0.22,1.912651,0.052744,0.359252,0.140724,0.252270,0.872851,0.596030,0.532133,...,1.093497,1.131531,1.227458,1.450936,1.383674,1.429930,1.591903,1.845899,1.868426,2.071472
4,0.15,0.23,-1.100744,-1.144692,1.079295,0.629496,0.442990,0.644495,0.692457,0.485183,...,1.040195,0.731144,0.810771,0.781476,0.463825,0.588291,0.543630,0.370510,0.642869,1.007751


In [34]:
#MIN ruta
# Hold out the last 60 days or so of the remaining data as a validation set
train_features, train_targets = features, targets
val_features, val_targets = test_features, test_targets

Here we can see one of the images.

In [8]:
image, label = next(iter(trainloader))
#helper.imshow(image[0,:]);

# Building the network
Here you should define your network. As with MNIST, each image is 28x28 which is a total of 784 pixels, and there are 10 classes. You should include at least one hidden layer. We suggest you use ReLU activations for the layers and to return the logits or log-softmax from the forward pass. It's up to you how many layers you add and the size of those layers.


In [8]:
# TODO: Define your network architecture here, each picture is 28*28 pixels

#This is my solution
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch import optim

model = nn.Sequential(nn.Linear(784, 256),
                      nn.ReLU(),
                      nn.Linear(256,128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

#criterion = nn.NLLLoss()
#optimizer = optim.SGD(model.parameters(), lr=0.003)

#get data
#dataiter = iter(trainloader)
#images, labels = next(dataiter)

#flatten images
#images = images.view(images.shape[0],-1)





In [9]:
from torch import nn, optim
import torch.nn.functional as F
#network architecture
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,64)
        self.fc4 = nn.Linear(64,10)
        
    def forward(self, x) :
        #flatten input tensor
        x = x.view(x.shape[0], -1) #no need to do this in input loop, batch size, automatic size for total elements
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x
    
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.003)

## Train the network
Now you should create your network and train it. First you'll want to define the criterion ( something like nn.CrossEntropyLoss) and the optimizer (typically optim.SGD or optim.Adam).

Then write the training code. Remember the training pass is a fairly straightforward process:

Make a forward pass through the network to get the logits
Use the logits to calculate the loss
Perform a backward pass through the network with loss.backward() to calculate the gradients
Take a step with the optimizer to update the weights
By adjusting the hyperparameters (hidden units, learning rate, etc), you should be able to get the training loss below 0.4.


In [35]:
#optimizer, stocastic gradient
#optimizer = optim.SGD(model.parameters(), lr = 0.01)

epoch = 5
for e in range (epoch):
    running_loss = 0
    #for image, label in trainloader:
    for instrument, AdjIt in features:    
        #flatten images
        #image = image.view(image.shape[0],-1)
                
        optimizer.zero_grad() 
        logProb = model.forward(instrument)  #pass in model(image) it will automatically run the forward method
        loss = criterion(logProb, AdjIt)
        loss.backward()
        optimizer.step()
    
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(features)}")

ValueError: too many values to unpack (expected 2)

In [6]:
# TODO: Create the network, define the criterion and optimizer
# TODO: Train the network here

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import helper

# Test out your network!

dataiter = iter(testloader)
images, labels = dataiter.next()
img = images[0]
# Convert 2D image to 1D vector
img = img.resize_(1, 784)

# TODO: Calculate the class probabilities (softmax) for img
ps = torch.exp(model(img))

# Plot the image and probabilities
helper.view_classify(img.resize_(1, 28, 28), ps, version='Fashion')